# En este notebook se calculara y guardará un AlphaSet

In [4]:
import sys
import os
import pickle
import time
import scipy.special as scsp

sys.path.append("src")
from src.AlphaSet import AlphaSet
import numpy as np

___

## Parámetros para el AlphaSet
- I: Dimensión del Movimiento Browniano
- J: Largo de los alphas que generan los Polinomios de Wick
- K: Grado máximo de los Polinomios de Wick
- n: Cantidad de divisiones de tiempo para el Movimiento Browniano
- T: Tiempo total del Movimiento Browniano
- NNormals: Cantidad de Normales (Omegas) que se van a generar para entrenar / evaluar el modelo

In [6]:
I, J, K, n, T, NNormals = 1, 50, 2, 100, 0.25, 20
batch_size = 2 # Para calcular en tandas
SEED = 69
pkl_file = f"files/alpha_set_I{I}_J{J}_K{K}_n{n}_T{T}.pkl"

___

## Generar el AlphaSet

In [5]:
# Método lento

np.random.seed(SEED)
alpha_set = AlphaSet(I, J, K, n, T)
alpha_set.calculate_alphas()
start_time = time.time()
for _ in range(NNormals):
    normal = np.random.normal(size=(I, J))
    alpha_set.normals.append(normal)
alpha_set.calculate_wick_values()
alpha_set.calculate_increments()
alpha_set.evaluate_paths()
elapsed_time = time.time() - start_time
print(f"Time taken for slow method: {elapsed_time:.2f} seconds")
alpha_set.save_alpha_set(pkl_file)

KeyboardInterrupt: 

In [7]:
# Cargar o crear
np.random.seed(SEED)
if os.path.exists(pkl_file):
    with open(pkl_file, "rb") as f:
        aset = pickle.load(f)
else:
    aset = AlphaSet(I, J, K, n, T)
    aset.calculate_alphas_fast()

# Bucle incremental
start = time.time()
while len(aset.normals) < NNormals:
    remaining = NNormals - len(aset.normals)
    b = min(batch_size, remaining)
    normals = np.random.normal(size=(b, I, J))
    aset.add_normals(normals)

    # checkpoint
    with open(pkl_file, "wb") as f:
        pickle.dump(aset, f)
    print(f"Guardadas {len(aset.normals)}/{NNormals}")
end = time.time() - start
print(f"Tiempo total: {end:.2f} segundos")
print("AlphaSet completo ✔")

Guardadas 7/20
Guardadas 9/20
Guardadas 11/20
Guardadas 13/20
Guardadas 15/20
Guardadas 17/20
Guardadas 19/20
Guardadas 20/20
Tiempo total: 0.22 segundos
AlphaSet completo ✔


___

In [ ]:
import time
import copy

I, J, K, n, T = 1, 15, 2, 50, 0.25
N_NORMALS     = 3
SEED          = 123
atol          = 1e-12

# ---------- 1. AlphaSet (lento: calculate_alphas original) -------
np.random.seed(SEED)
slow = AlphaSet(I, J, K, n, T)

t0 = time.perf_counter()
slow.calculate_alphas()          #  ← versión exhaustiva original
print(f"Tiempo slow  : {time.perf_counter()-t0:.3f} s")

# ---------- 2. AlphaSet (rápido: nueva función) ------------------
np.random.seed(SEED)
fast = AlphaSet(I, J, K, n, T)

t0 = time.perf_counter()
fast.calculate_alphas_fast()     #  ← tu función optimizada
print(f"Tiempo fast  : {time.perf_counter()-t0:.3f} s")

# ---------- 3. Comprobar que los α son los mismos ---------------
def alphas_to_set(alpha_dict):
    return {
        k: {tuple(a.values.flatten()) for a in lst}
        for k, lst in alpha_dict.items()
    }

same_alphas = alphas_to_set(slow.alphas) == alphas_to_set(fast.alphas)
print("Índices α idénticos:", same_alphas)

# ---------- 4. Continuamos pipeline para corroborar todo ---------
normals = np.random.normal(size=(N_NORMALS, I, J))
slow.add_normals(copy.deepcopy(normals))
fast.add_normals(copy.deepcopy(normals))

def deep_equal(a, b):
    if isinstance(a, list):
        return len(a) == len(b) and all(deep_equal(x, y) for x, y in zip(a, b))
    if isinstance(a, dict):
        return a.keys() == b.keys() and all(deep_equal(a[k], b[k]) for k in a)
    return np.allclose(np.asarray(a), np.asarray(b), atol=atol, rtol=0)

ok = (
    deep_equal(slow.wick_values,         fast.wick_values)         and
    deep_equal(slow.brownian_increments, fast.brownian_increments) and
    deep_equal(slow.brownian_paths,      fast.brownian_paths)
)

print("\n================ RESULTADO ================")
print("¡Métodos equivalentes! ✅" if (same_alphas and ok) else "❌  Hay diferencias")


Tiempo slow  : 1.041 s
Tiempo fast  : 0.003 s
Índices α idénticos: True

================ RESULTADO ================
¡Métodos equivalentes! ✅


In [4]:
# ---------------- PARÁMETROS ----------------
I, J, K, n, T    = 1, 16, 2, 100, 0.25
NNormals         = 5          # número total de normales a precalcular
batch_size       = 2          # tamaño de lote para el método rápido
SEED             = 69         # semilla para reproducibilidad
pkl_file = f"files/alpha_set_I{I}_J{J}_K{K}_n{n}_T{T}.pkl"
atol_compare     = 1e-12      # tolerancia de np.allclose
# -----------------------------------------------------------


# ---------- 1. AlphaSet "clásico" --------------------------
np.random.seed(SEED)
classic = AlphaSet(I, J, K, n, T)
classic.calculate_alphas()

for _ in range(NNormals):
    classic.normals.append(np.random.normal(size=(I, J)))
classic.calculate_wick_values()
classic.calculate_increments()
classic.evaluate_paths()
print("AlphaSet clásico listo")


# ---------- 2. AlphaSet "rápido / incremental" ------------
np.random.seed(SEED)                       # MISMA semilla
if os.path.exists(pkl_file):
    with open(pkl_file, "rb") as f:
        fast = pickle.load(f)
    print("Checkpoint encontrado.  Normales actuales:", len(fast.normals))
else:
    fast = AlphaSet(I, J, K, n, T)
    fast.calculate_alphas()
    print("AlphaSet rápido creado de cero")

while len(fast.normals) < NNormals:
    remaining = NNormals - len(fast.normals)
    b = min(batch_size, remaining)
    normals = np.random.normal(size=(b, I, J))
    fast.add_normals(normals)

    with open(pkl_file, "wb") as f:
        pickle.dump(fast, f)
    print(f"Guardadas {len(fast.normals)}/{NNormals}")

print("AlphaSet rápido completo ✔")


# ---------- 3. Función de comparación ----------------------
def compare(a, b, name, atol=atol_compare):
    """
    Compara recursivamente listas, diccionarios y arrays.
    Devuelve True si son iguales (hasta atol) y False en caso contrario.
    """
    if isinstance(a, list):
        if len(a) != len(b):
            print(f"{name}: len diferente ({len(a)} vs {len(b)})")
            return False
        return all(compare(x, y, f"{name}[{i}]", atol) for i, (x, y) in enumerate(zip(a, b)))

    if isinstance(a, dict):
        if a.keys() != b.keys():
            print(f"{name}: claves distintas")
            return False
        return all(compare(a[k], b[k], f"{name}['{k}']", atol) for k in a)

    # caso base: array / escalar
    eq = np.allclose(np.asarray(a), np.asarray(b), rtol=0, atol=atol)
    if not eq:
        diff = np.max(np.abs(np.asarray(a) - np.asarray(b)))
        print(f"{name}: no coincide (max |Δ| = {diff})")
    return eq


# ---------- 4. LLAMAR A LA COMPARACIÓN ---------------------
ok = (
    compare(classic.normals,             fast.normals,             "normals") and
    compare(classic.wick_values,         fast.wick_values,         "wick_values") and
    compare(classic.brownian_increments, fast.brownian_increments, "increments") and
    compare(classic.brownian_paths,      fast.brownian_paths,      "paths")
)

print("\n==================== RESULTADO ====================")
print("AlphaSets equivalentes ✔✔" if ok else "¡Los AlphaSets difieren!")

AlphaSet clásico listo
AlphaSet rápido creado de cero
Guardadas 2/5
Guardadas 4/5
Guardadas 5/5
AlphaSet rápido completo ✔

==================== RESULTADO ====================
AlphaSets equivalentes ✔✔
